# 공용 함수
- 수정하지 않는 것을 추천드립니다

In [37]:
from jy import get_all, get_scaled
import pandas as pd
import plotly.express as px
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")

In [38]:
# 상대경로 지정 없이 원본 csv 임포트
main_df = get_all()
scaled_df = get_scaled(0) #0 = sd, 1 = nm, 2 = mm, 3 = rb
main_df_columns = main_df.columns
scaled_columns = scaled_df.columns
date_df = main_df.loc[:,"date"]

In [39]:
# 각 지역구와 관련된 변수들
# common은 지역 구분 없는 전체 변수
gw_vars_scaled = scaled_df.loc[:,scaled_columns[1:8]]
ddm_vars_scaled = scaled_df.loc[:,scaled_columns[8:15]]
sd_vars_scaled = scaled_df.loc[:,scaled_columns[15:22]]
jl_vars_scaled = scaled_df.loc[:,scaled_columns[22:29]]
common_vars_scaled = scaled_df.loc[:,scaled_columns[29:45]]

In [40]:
# 각 지역구와 관련된 변수들
# common은 지역 구분 없는 전체 변수
gw_vars = main_df.loc[:,main_df_columns[2:9]]
ddm_vars = main_df.loc[:,main_df_columns[9:16]]
sd_vars = main_df.loc[:,main_df_columns[16:23]]
jl_vars = main_df.loc[:,main_df_columns[23:30]]
common_vars = main_df.loc[:,main_df_columns[30:46]]

In [41]:
# 각 지역구의 따릉이 수요량 
gw_demand=main_df.loc[:,main_df_columns[46]]
ddm_demand=main_df.loc[:,main_df_columns[47]]
sd_demand=main_df.loc[:,main_df_columns[48]]
jl_demand=main_df.loc[:,main_df_columns[49]]

In [42]:
# 날짜와 수요량 컬럼을 원하는 대로 정리해주는 함수
def create_y (df_demand, date_name, y_name):
    result = pd.concat([date_df, df_demand], axis=1)
    result = result.rename(columns={"date":date_name, result.columns[1]:y_name})
    return result

In [43]:
# 변수들을 일부 샘플링한 dataframe의 리스트를 반환하는 함수
## amount는 (수요량 이외에) 추가로 샘플링할 변수의 개수 (0, 1, 2 가능. 0은 수요량만 반환)
## df_y는 날짜와 수요량이 포함된 데이터프레임. create_y 함수로 정제한 값을 넣으면 됩니다
## df_list는 추가 변수로 사용할 모든 데이터프레임들의 리스트. ~~~_vars 중 원하는 것을 리스트로 넣으면 됩니다
    ## 예시) 광진구 + 공용 변수의 경우: df_list=[gw_vars, comon_vars]
## 반환하는 값은 튜플 {"data": 데이터프레임, "name":포함된 컬럼명의 리스트} 로 이루어진 리스트입니다
    
def sample_multivariate(amount, df_y, df_list):
    samples = []
    # 데이터프레임을 하나로 통합
    df = pd.concat(df_list, axis=1)
    if amount == 0:
        # y값만 리턴
        samples.append({"data":pd.concat([df_y], axis=1),"name":[]})
    elif amount == 1:
        # sample을 한 column씩 추출
        for index in range(len(df.columns)):
            sample = pd.concat([df_y,df.loc[:,df.columns[index]]],axis=1)
            samples.append({"data":sample, "name":[df.columns[index]]})
    elif amount == 2:
        # sample을 두 column씩 추출
        for first_index in range(len(df.columns)):
            remaining_columns = df.columns[first_index+1:]
            for second_index in range(len(remaining_columns)):
                targets = [df.columns[first_index],remaining_columns[second_index]]
                sample = pd.concat([df_y,df.loc[:,targets]],axis=1)
                samples.append({"data":sample, "name":targets})
    return samples
    

In [44]:
# 데이터를 자르는 함수
# 일단은 2021년 1월 1일 기준으로 자르게 일반화 해놓았습니다

def split_sample(df):
    train = df.loc[:1095]
    test = df.loc[1096:]
    return {"train":train, "test":test}


In [45]:
def create_result(indexes):
    return pd.DataFrame(index=["Column"] + indexes)

공용 함수 End

In [46]:
gw_y = create_y(gw_demand, "ds","y")

In [47]:
selected  = gw_vars.drop(["pop_GW","station_GW"],axis=1)


In [48]:
test_dfs = sample_multivariate(2,gw_y,[selected])
test_dfs[0]

{'data':               ds      y  air_GW  real_pop_GW
 0       1/1/2018  0.592    42.0      421.214
 1       1/2/2018  0.840    43.0      408.056
 2       1/3/2018  0.828    34.0      406.174
 3       1/4/2018  0.792    44.0      409.168
 4       1/5/2018  0.818    64.0      408.793
 ...          ...    ...     ...          ...
 1456  12/27/2021  3.830    23.0      373.361
 1457  12/28/2021  4.510    42.0      372.840
 1458  12/29/2021  4.490    54.0      372.609
 1459  12/30/2021  4.444    26.0      375.444
 1460  12/31/2021  3.616    21.0      373.701
 
 [1461 rows x 4 columns],
 'name': ['air_GW', 'real_pop_GW']}

In [49]:
# create_result 함수로 result 생성
result = create_result(["MAE","MAE_val","Metrics"])
# result가 잘 생성되었는 지 확인


In [50]:
# for문을 통해 모델 생성
for index, test_df in enumerate(test_dfs):

    # 샘플링 된 컬럼명
    columns = test_df["name"]

    # test_df로부터 test, train 데이터셋 분리
    splitted = split_sample(test_df["data"])
    test = splitted["test"]
    train = splitted["train"]

    # 모델 생성
    m = NeuralProphet(
        growth='off', # 추세 유형 설정(linear, discontinuous, off 중 선택 가능)
        yearly_seasonality=True, #년간 계절성 설정
        weekly_seasonality=True, #주간 계절성 설정
        daily_seasonality=True, #일간 계절성 설정
        batch_size=32,#배치 사이즈 설정
        epochs=30,#학습 횟수 설정
        learning_rate=0.1, # 학습률 설정
    )
    # 독립 변인(변수) 추가 및 정규화
    # regressor의 names로는 샘플링된 컬럼명인 columns를 사용했습니다
    m = m.add_lagged_regressor(names=columns, normalize="minmax") 
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # 학습 수행
    metrics = m.fit(train, freq='D', validation_df=test)
    
    # 기록
    # result의 첫 행은 Column 명입니다!
    result[index] = [columns,metrics.loc[29,"MAE"],metrics.loc[29,"MAE_val"],metrics]





Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [51]:
result

,0,1,2,3,4,5,6,7,8,9
Column,"[air_GW, real_pop_GW]","[air_GW, pop_male_GW]","[air_GW, pop_female_GW]","[air_GW, mvpop_GW]","[real_pop_GW, pop_male_GW]","[real_pop_GW, pop_female_GW]","[real_pop_GW, mvpop_GW]","[pop_male_GW, pop_female_GW]","[pop_male_GW, mvpop_GW]","[pop_female_GW, mvpop_GW]"
MAE,1.632146,1.485952,1.542874,1.836879,1.518629,1.543294,1.631272,1.49364,1.446335,1.493232
MAE_val,2.130268,2.241436,1.946627,2.763425,2.165007,1.934463,2.044532,2.539677,2.195533,2.02251
Metrics,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_val...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_va...,MAE_val RMSE_val Loss_val RegLoss_val...


In [57]:
px.line(result.loc["Metrics"][2],y=["MAE","MAE_val"],width=800,height=400)

In [58]:
px.line(result.loc["Metrics"][5],y=["MAE","MAE_val"],width=800,height=400)

In [59]:
px.line(result.loc["Metrics"][6],y=["MAE","MAE_val"],width=800,height=400)

In [60]:
px.line(result.loc["Metrics"][9],y=["MAE","MAE_val"],width=800,height=400)